# Explaining the Chess Game Analysis Pipeline

The Chess Game Analysis Pipeline is designed to process and analyze chess games from a PGN (Portable Game Notation) file. This pipeline is particularly tailored for filtering and categorizing games based on specific criteria such as time control and player ELO ratings. Below is an overview of its components and functionalities.

## Overview

The pipeline is structured around a `Pipeline` struct in Julia, which is used to manage the processing of chess games. It includes various functions for counting games, reading specific games, generating random game indices, and more.

## Global Variables

- `filename`: The path to the PGN file containing the chess games.
- `seed`: A seed value for controlling randomness in the pipeline, ensuring reproducible results.

## The `Pipeline` Struct

The `Pipeline` struct is the core of the pipeline, containing fields and methods necessary for processing the chess games.

### Fields

- `filename`: Path to the PGN file.
- `seed`: Seed for random number generation, ensuring consistency.
- `games_per_cell`: The target number of games for each category in the analysis matrix.
- `matrix`: A 3x3 matrix, each cell containing an array of integers representing game indices.
- `total_games`: Total number of games in the file.

### Methods

- `count_games_in_file`: Counts the total number of games in the PGN file.
- `read_nth_game`: Reads the nth game from the file.
- `random_arr`: Generates a random number within the range of total games in the file.
- `write_file`: Writes game data to a file, either creating a new file or appending to an existing one.
- `check_criteria`: Checks whether a game meets the specified criteria based on time controls and player ELO ranges.

## Processing Procedure

1. **Initialization**: A `Pipeline` instance is created with the specified filename and seed.
2. **Counting Games**: The total number of games in the file is counted.
3. **Game Selection**: Games are randomly selected and checked against specified criteria.
4. **Criteria Checking**: Each game is evaluated to determine if it fits the desired time control and ELO range categories.
5. **Data Storage**: Valid games are stored in the matrix, categorized based on the analysis goals.


# Imports

In [2]:
# import julia libraries
using PyCall
using Random
using ProgressMeter
# using DataStructures
# using Pandas

# import python libraries
# np = pyimport("numpy")

# if "invalid redefinition of constant chess" error pops out
# try to clear cache and restart the session
@pyimport chess
cp = pyimport("chess.pgn")

PyObject <module 'chess.pgn' from '/home/ubuntu/.local/lib/python3.10/site-packages/chess/pgn.py'>

In [3]:
struct Pipeline
    filename::String        # Filename of the total large file
    export_path::String     # The folder of the exporting files
    seed::Int               # Seed for the randomness control
    total_games::Int        # Total number of games in the file
    games_per_cell::Int     # Number of games in each cell in the table
    round_draw_numbers::Int # Number of random indices to draw each round
    cells::Matrix{Int}      # 2D matrix to track the number of games in each cell

    function Pipeline(filename::String, export_path::String, seed::Int, total_games::Int, games_per_cell::Int, round_draw_numbers::Int)
        new(filename, export_path, seed, total_games, games_per_cell, round_draw_numbers, fill(0, 3, 3))
    end
end


## File and group of games operation

In [4]:
function random_arr(pipeline::Pipeline, start_nth::Int, n::Int)::Vector{Int32}
    # 1-indexed
    Random.seed!(pipeline.seed)
    rand(1:pipeline.total_games, start_nth - 1)
    return rand(1:pipeline.total_games, n)
end

random_arr (generic function with 1 method)

In [5]:
function write_file(filename::String, game)
    
    # Check if the file exists
    file_mode = isfile(filename) ? "a" : "w"

    # Open the file with appropriate mode
    open(filename, file_mode) do new_pgn
        # Create an exporter instance
        exporter = cp.FileExporter(new_pgn)
        
        # Use the game.accept method to write the game to the file
        game.accept(exporter)
    end
end


write_file (generic function with 1 method)

## Single game operation

In [5]:
# helper method of check_criteria()
function time_and_elo(headers)
    time_control = get(headers, "TimeControl")
    WhiteElo = parse(Int64, get(headers, "WhiteElo"))
    BlackElo = parse(Int64, get(headers, "BlackElo"))

    return time_control, WhiteElo, BlackElo
end

time_and_elo (generic function with 1 method)

In [6]:
function check_criteria(game)
    # criteria (will set as parameter in future)
    valid_time = ["180+0", "600+0", "1800+0"]
    elo_ranges = [(1250, 1350), (1750, 1850), (2150, Inf)]

    # time groups:
    # 0 - Invalid time group
    # 1 - Blitz: 180+0 seconds (3 minutes)
    # 2 - Rapid: 600+0 seconds (10 minutes)
    # 3 - Classical: 1800+0 seconds (30 minutes)

    time_group = 0

    # valid elo groups:
    # 0 - Invalid time group
    # 1 - Beginners: 1250-1350
    # 2 - Intermediate: 1750-1850
    # 3 - Expert: Above 2150

    elo_group = 0

    # get data from the game header
    time_control, player1_elo, player2_elo = time_and_elo(game.headers)

    for time in 1:length(valid_time)
        if (cmp(valid_time[time], time_control) == 0)
            time_group = time
        end
    end

    for elo in 1:length(elo_ranges)
        if ((elo_ranges[elo][1] < player1_elo) && (player1_elo < elo_ranges[elo][2]) && (elo_ranges[elo][1] < player2_elo) && (player2_elo < elo_ranges[elo][2]))
            elo_group = elo
        end
    end

    return time_group, elo_group
end

check_criteria (generic function with 1 method)

In [7]:
function game_cell_filename(criteria_result)::String
    time_group, elo_group = criteria_result

    # Mapping time groups to their string representations
    time_control = if time_group == 1
        "blitz"
    elseif time_group == 2
        "rapid"
    elseif time_group == 3
        "classical"
    end

    # Mapping ELO groups to their string representations
    elo_range = if elo_group == 1
        "beginner"
    elseif elo_group == 2
        "intermediate"
    elseif elo_group == 3
        "expert"
    end

    filename = "$(time_control)_$(elo_range).pgn"

    return filename
end


game_cell_filename (generic function with 1 method)

1~93218629 choose 100,000 sorted in order then all the way through the list and then stop
check if cells are full
if full, close so don't fill them in in next round
repeat the process, until all cells are closed (leave cells over-filled)


after all cells are closed
<!-- then random select certain amount of the games  in cell? -->
leave cells over-filled


random draw without replacement

In [8]:
function loop_workflow(pipeline::Pipeline, cell_status::Matrix{Bool}, random_indices::Vector{Int32})::Pipeline
    # Initialize indexes
    index = 1 # current file reading process
    rand_in = 1 # the randomed arr index

    open(pipeline.filename, "r") do pgn

        # Check sizes
        seekend(pgn)
        fileSize = position(pgn)
        seekstart(pgn)
        indices_size = length(random_indices)
        
        # Initialize progress bar
        file_scaning_progress = Progress(fileSize; dt=1.0, desc="Current Round Progress...")
        fetch_arr_game_progress = Progress(indices_size; dt=1.0, desc="Selected game in array...")

        while !eof(pgn)
            game = cp.read_game(pgn)
            index += 1

            # Update file progress bar
            ProgressMeter.update!(file_scaning_progress, position(pgn))

            # in case of overflow error
            if rand_in > length(random_indices)
                continue
            end

            # check if the game is the chosen one
            if index != random_indices[rand_in]
                continue
            end

            # increment the random arr index by 1
            rand_in += 1

            # Update random arr index progress bar
            ProgressMeter.update!(fetch_arr_game_progress, rand_in)

            # Check the game's criteria to see if the game satisfies the condition
            criteria_result = check_criteria(game)

            # If the game satisfies the condition
            if all(x -> x != 0, criteria_result)
                if !cell_status[criteria_result...]
                    filename = game_cell_filename(criteria_result)
                    write_file(joinpath(pipeline.export_path, filename), game)
                    pipeline.cells[criteria_result...] += 1
                end
            end

        end

        # rand_in -= 1
    
    end
    
    println("Total number of game in file: $index");
    # println("Index of last game read in file $(random_indices[rand_in]), the $rand_in th number of this randomed arr");
    
    return pipeline

end

loop_workflow (generic function with 1 method)

In [9]:
function main(pipeline::Pipeline)
    # Initialize a 3x3 boolean matrix for cell status (false = open, true = closed)
    cell_status = fill(false, 3, 3)

    current_index = 1
    drawn_numbers = Set{Int32}()

    # While not all cells are closed
    while any(!cell for cell in cell_status)
        # Start a new round
        println("----------------------------—----------------------------—")
        println("Start a new round, seed drawing number from $current_index")
        println("----------------------------—----------------------------—")
        
        # Call the random_arr from 1 to the total games in the file 
        # to get 100,000 random numbers as the game index
        # also time it
        @time new_indices = unique(sort(random_arr(pipeline, current_index, pipeline.round_draw_numbers)))

        # Filter out indices that have already been drawn
        random_indices = filter(idx -> !(idx in drawn_numbers), new_indices)
        if isempty(random_indices)
            println("random indices is 0, quiting the program...")
            return
        end

        println("Finish drawing and sorting process")
        
        # Add the new indices to the drawn numbers set
        union!(drawn_numbers, random_indices)

        println("Start loop workflow...")
        # read and write games in the loop workflow and then update pipeline cells
        pipeline = loop_workflow(pipeline, cell_status, random_indices)

        # Check if each cell reaches or exceeds the condition (e.g., 3000)
        # If so, mark the cell as closed
        for i in 1:size(pipeline.cells, 1)
            for j in 1:size(pipeline.cells, 2)
                if pipeline.cells[i, j] >= pipeline.games_per_cell
                    cell_status[i, j] = true
                end
            end
        end

        # size for report
        random_indices_size = length(random_indices)
        drawn_numbers_size = length(drawn_numbers)

        println("Drew unique number $random_indices_size this round, current total drawn number size: $drawn_numbers_size")
        current_index += pipeline.round_draw_numbers
    end

    println("Work finished! I know, finally! (but it is just the first pipeline...)");
    # After all cells are closed, leave cells over-filled
    # Additional random draw logic (if needed) goes here
end


main (generic function with 1 method)

# Define variables

In [6]:
# File name of the original decompressed file
filename = "./data/lichess_db_standard_rated_2023-09.pgn"

# The seed for the randomness control in this project
seed = 2023

# Export path
export_path = "./data/exported/"

# Total games
total_games = 93218629

# Games per cell
games_per_cell = 3000

# Number of random indices to draw each round
round_draw_numbers = 100000

# Pipeline struct
# Initialize with filename, export_path, seed, total_games, and games_per_cell, round_draw_numbers
pipeline = Pipeline(filename, export_path, seed, total_games, games_per_cell, round_draw_numbers)

Pipeline("./data/lichess_db_standard_rated_2023-09.pgn", "./data/exported/", 2023, 93218629, 3000, 100000, [0 0 0; 0 0 0; 0 0 0])

In [11]:
# main(pipeline)

# Test code

In [7]:
# File name of the original decompressed file
test_filename = "./test10000.pgn"

# The seed for the randomness control in this project
test_seed = 2026

# Export path
test_export_path = "./data/test_exported/"

# Total games
test_total_games = 10000

# Games per cell
test_games_per_cell = 1

# Number of random indices to draw each round
test_round_draw_numbers = 1000

# Pipeline struct
# Initialize with filename, export_path, seed, total_games, and games_per_cell, round_draw_numbers
test_pipeline = Pipeline(test_filename, test_export_path, test_seed, test_total_games, test_games_per_cell, test_round_draw_numbers)

Pipeline("./test10000.pgn", "./data/test_exported/", 2026, 10000, 1, 1000, [0 0 0; 0 0 0; 0 0 0])

In [8]:
# @time main(test_pipeline)

In [16]:
# @time begin
#     # Keep drawing until we reach the required number of unique indices
#     while length(new_indices) < pipeline.round_draw_numbers
#         println("Current new_indices is $(length(new_indices)), Drawing...")
#         # Draw random indices
#         potential_new_indices = random_arr(pipeline, current_index, pipeline.round_draw_numbers - length(new_indices))

#         # Add them to the set of new indices, ensuring uniqueness
#         union!(new_indices, potential_new_indices)

#         # Check if we have reached the total number of games
#         if length(drawn_numbers) + length(new_indices) >= pipeline.total_games
#             # Draw only the remaining undrawn games
#             remaining_indices = setdiff(Set(1:pipeline.total_games), drawn_numbers)
#             union!(new_indices, remaining_indices)
#             break
#         end
#     end
# end

# # Convert the set to a vector and sort
# random_indices = sort(collect(new_indices))

# # Filter out indices that have already been drawn
# random_indices = filter(idx -> !(idx in drawn_numbers), random_indices)

# if isempty(random_indices)
#     println("random indices is 0, quiting the program...")
#     return
# end
